# 29차시: [실습] ChatGPT API와 재무제표 요약 및 분석

## 학습 목표
- 네이버 금융에서 실제 재무제표 데이터 수집
- OpenAI / Gemini API를 활용한 재무제표 분석 실습
- 프롬프트 엔지니어링으로 분석 품질 향상

## 학습 내용
1. 네이버 금융에서 재무 데이터 수집
2. 데이터 전처리 및 포맷팅
3. 프롬프트 설계
4. ChatGPT / Gemini API로 분석
5. 기업 비교 분석

## 이전 차시 연계
- 15차시: 네이버 금융 크롤링 (`pd.read_html`)
- 27차시: 프롬프트 엔지니어링
- 28차시: ChatGPT, Gemini API 설정

In [1]:
# !pip install openai google-genai python-dotenv requests beautifulsoup4 lxml

In [2]:
import os
import json
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from IPython.display import display, Markdown, HTML
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# Colab에서 .env 파일 업로드
try:
    from google.colab import files
    print("[Colab 환경 감지]")
    print("=" * 60)
    print(".env 파일을 업로드해주세요.")
    print()
    uploaded = files.upload()
    load_dotenv('.env')
except ImportError:
    print("[로컬 환경]")
    print("=" * 60)
    print(".env 파일이 현재 디렉토리에 있어야 합니다.")

[Colab 환경 감지]
.env 파일을 업로드해주세요.



Saving .env to .env (1)


---
## API 클라이언트 설정

28차시에서 설정한 OpenAI와 Gemini API를 모두 사용할 수 있습니다.

In [3]:
# API 키 확인
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
GEMINI_API_KEY = os.getenv('GOOGLE_API_KEY')

print(f"키: {OPENAI_API_KEY[:10]}...")
print(f"키: {GEMINI_API_KEY[:10]}...")

키: sk-FeQOBhF...
키: AIzaSyBARt...


In [17]:
# 사용할 API 선택 (openai 또는 gemini)
USE_API = "gemini"  # "openai" 또는 "gemini"

---
## 1. 네이버 금융에서 재무 데이터 수집

15차시에서 배운 `pd.read_html()`을 활용하여 네이버 금융에서 실제 재무제표를 수집합니다.

In [18]:
# 공통 헤더 설정
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
}

def get_company_name(stock_code):
    """
    종목코드(stock_code)를 입력받아
    네이버 금융 페이지에서 회사명을 크롤링하여 반환하는 함수
    """

    url = f"https://finance.naver.com/item/main.nhn?code={stock_code}"

    try:
        # HTTP GET 요청을 보내 HTML 페이지 가져오기
        response = requests.get(url, headers=HEADERS)

        soup = BeautifulSoup(response.text, 'html.parser')

        # 회사명이 위치한 HTML 태그 선택
        name_tag = soup.select_one('div.wrap_company h2 a')

        # 회사명 태그가 존재하면 텍스트(회사명) 반환
        if name_tag:
            return name_tag.text.strip()

        # 회사명을 찾지 못한 경우 종목코드 그대로 반환
        return stock_code

    except Exception:
        return stock_code


def get_financial_summary(stock_code):
    """네이버 금융에서 재무 요약 정보 수집"""
    url = f"https://finance.naver.com/item/main.nhn?code={stock_code}"

    try:
        # 모든 테이블 읽기
        tables = pd.read_html(url, encoding='euc-kr')

        # 재무 요약 테이블 찾기 (보통 '매출액' 포함)
        financial_table = None
        for table in tables:
            table_str = str(table.columns) + str(table.values)
            if '매출액' in table_str or '영업이익' in table_str:
                financial_table = table
                break

        if financial_table is None:
            return None

        # 회사명 조회
        company_name = get_company_name(stock_code)

        return {
            'company_name': company_name,
            'stock_code': stock_code,
            'table': financial_table
        }
    except Exception as e:
        print(f"[오류] {stock_code} 데이터 수집 실패: {e}")
        return None

In [19]:
# 삼성전자 재무 데이터 수집
print("[네이버 금융에서 재무 데이터 수집]")
print("=" * 60)

samsung_data = get_financial_summary("005930")

if samsung_data:
    print(f"\n회사명: {samsung_data['company_name']}")
    print(f"종목코드: {samsung_data['stock_code']}")
    print("\n[재무 요약 테이블]")
    display(samsung_data['table'])
else:
    print("데이터 수집 실패")

[네이버 금융에서 재무 데이터 수집]

회사명: 삼성전자
종목코드: 005930

[재무 요약 테이블]


주요재무정보    최근 연간 실적                                       최근 분기 실적  \
       주요재무정보     2022.12     2023.12     2024.12  2025.12(E)    2024.09   
       주요재무정보      IFRS연결      IFRS연결      IFRS연결      IFRS연결     IFRS연결   
0         매출액  3022314.00  2589355.00  3008709.00  3291027.00  790987.00   
1        영업이익   433766.00    65670.00   327260.00   401605.00   91834.00   
2       당기순이익   556541.00   154871.00   344514.00   398255.00  101009.00   
3       영업이익률       14.35        2.54       10.88       12.20      11.61   
4        순이익률       18.41        5.98       11.45       12.10      12.77   
5   ROE(지배주주)       17.07        4.15        9.03        9.53       8.79   
6        부채비율       26.41       25.36       27.93         NaN      27.19   
7        당좌비율      211.68      189.46      187.80         NaN     190.56   
8         유보율    38144.29    39114.28    41772.84         NaN   41198.62   
9      EPS(원)     8057.00     2131.00     4950.00     5727.00    1440.00   
10     PER(배)        6.86       36.84       10.75       22.44      13.03   
11     BPS(원)    50817.00    52002.00    57981.00    63204.00   55376.00   
12     PBR(배)        1.09        1.51        0.92        2.03       1.11   
13   주당배당금(원)     1444.00     1444.00     1446.00     1527.00     361.00   
14   시가배당률(%)        2.61        1.84        2.72         NaN       0.59   
15    배당성향(%)       17.92       67.78       29.18         NaN      25.07   

                                                           
      2024.12    2025.03    2025.06    2025.09 2025.12(E)  
       IFRS연결     IFRS연결     IFRS연결     IFRS연결     IFRS연결  
0   757883.00  791405.00  745663.00  860617.00  895006.00  
1    64927.00   66853.00   46761.00  121661.00  167122.00  
2    77544.00   82229.00   51164.00  122257.00  156220.00  
3        8.57       8.45       6.27      14.14      18.67  
4       10.23      10.39       6.86      14.21      17.46  
5        9.03       9.24       7.95       8.37        NaN  
6       27.93      26.99      26.36      26.64        NaN  
7      187.80     187.68     190.87     204.62        NaN  
8    41772.84   42056.84   42340.19   43418.06        NaN  
9     1115.00    1186.00     733.00    1783.00    2290.00  
10      10.75      11.20      13.36      17.42      56.13  
11   57981.00   59059.00   58135.00   60632.00   63204.00  
12       0.92       0.98       1.03       1.38       2.03  
13     363.00     365.00     367.00     370.00        NaN  
14       0.68       0.63       0.61       0.44        NaN  
15      32.40      30.48      49.73      20.43        NaN

---
## 2. 데이터 전처리 및 포맷팅

수집한 테이블 데이터를 LLM 분석에 적합한 형태로 변환합니다.

In [20]:
def format_for_llm(raw_data):
    """수집한 데이터를 LLM 입력용 텍스트로 변환"""
    if raw_data is None:
        return "데이터가 없습니다."

    lines = []
    lines.append(f"회사명: {raw_data['company_name']}")
    lines.append(f"종목코드: {raw_data['stock_code']}")
    lines.append(f"데이터 출처: 네이버 금융")
    lines.append(f"수집 일시: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
    lines.append("\n[재무제표 데이터]")
    lines.append(raw_data['table'].to_string())

    # 리스트에 저장된 모든 줄을 개행 문자로 결합하여 하나의 문자열로 반환
    return "\n".join(lines)

In [21]:
# 데이터 변환
samsung_text = format_for_llm(samsung_data)

print("[LLM 입력용 텍스트]")
print("=" * 60)
print(samsung_text)

[LLM 입력용 텍스트]
회사명: 삼성전자
종목코드: 005930
데이터 출처: 네이버 금융
수집 일시: 2026-01-09 12:19

[재무제표 데이터]
       주요재무정보    최근 연간 실적                                       최근 분기 실적                                                       
       주요재무정보     2022.12     2023.12     2024.12  2025.12(E)    2024.09    2024.12    2025.03    2025.06    2025.09 2025.12(E)
       주요재무정보      IFRS연결      IFRS연결      IFRS연결      IFRS연결     IFRS연결     IFRS연결     IFRS연결     IFRS연결     IFRS연결     IFRS연결
0         매출액  3022314.00  2589355.00  3008709.00  3291027.00  790987.00  757883.00  791405.00  745663.00  860617.00  895006.00
1        영업이익   433766.00    65670.00   327260.00   401605.00   91834.00   64927.00   66853.00   46761.00  121661.00  167122.00
2       당기순이익   556541.00   154871.00   344514.00   398255.00  101009.00   77544.00   82229.00   51164.00  122257.00  156220.00
3       영업이익률       14.35        2.54       10.88       12.20      11.61       8.57       8.45       6.27      14.14      18.67
4        순이익률   

---
## 3. 프롬프트 설계

27차시에서 배운 프롬프트 엔지니어링을 활용하여 재무 분석용 프롬프트를 설계합니다.

In [22]:
def create_analysis_prompt(financial_text):
    """
    재무제표 분석용 시스템 프롬프트와 사용자 프롬프트를 생성하는 함수
    - 분석 유형 분기 제거
    - 항상 종합 재무 분석 프롬프트를 반환
    """

    # 시스템 프롬프트: LLM의 역할과 분석 기준 정의
    system_prompt = """
당신은 15년 경력의 CFA 자격증을 보유한 증권 애널리스트입니다.
기업 재무제표를 분석하여 투자자에게 유용한 인사이트를 제공합니다.
분석은 객관적 데이터에 기반하며, 긍정적/부정적 요소를 균형 있게 제시합니다.
한국어로 답변합니다.
"""

    # 사용자 프롬프트: 실제 분석 요청 내용
    user_prompt = f"""
다음은 네이버 금융에서 수집한 재무제표입니다. 이를 기반으로 종합 분석해주세요.

{financial_text}

다음 형식으로 분석 결과를 작성해주세요:

## 1. 수익성 분석
- 매출액, 영업이익, 당기순이익 추이 평가
- 영업이익률 및 순이익률 수준과 변화

## 2. 안정성 분석
- 부채비율, 유동비율 등 주요 재무 안정성 지표
- 재무 구조의 안정성 평가

## 3. 성장성 분석
- 매출 및 이익 성장 추이
- 중장기 성장 가능성 평가

## 4. 투자 포인트
- 강점 2~3가지
- 리스크 요인 2~3가지

## 5. 종합 의견
- 투자 관점에서 100자 이내 요약
"""

    return system_prompt, user_prompt

In [23]:
# 프롬프트 확인
system_prompt, user_prompt = create_analysis_prompt(samsung_text)
print("[시스템 프롬프트]")
print("=" * 60)
print(system_prompt)
print("\n[사용자 프롬프트 - 일부]")
print("=" * 60)
print(user_prompt[:1000] + "...")

[시스템 프롬프트]

당신은 15년 경력의 CFA 자격증을 보유한 증권 애널리스트입니다.
기업 재무제표를 분석하여 투자자에게 유용한 인사이트를 제공합니다.
분석은 객관적 데이터에 기반하며, 긍정적/부정적 요소를 균형 있게 제시합니다.
한국어로 답변합니다.


[사용자 프롬프트 - 일부]

다음은 네이버 금융에서 수집한 재무제표입니다. 이를 기반으로 종합 분석해주세요.

회사명: 삼성전자
종목코드: 005930
데이터 출처: 네이버 금융
수집 일시: 2026-01-09 12:19

[재무제표 데이터]
       주요재무정보    최근 연간 실적                                       최근 분기 실적                                                       
       주요재무정보     2022.12     2023.12     2024.12  2025.12(E)    2024.09    2024.12    2025.03    2025.06    2025.09 2025.12(E)
       주요재무정보      IFRS연결      IFRS연결      IFRS연결      IFRS연결     IFRS연결     IFRS연결     IFRS연결     IFRS연결     IFRS연결     IFRS연결
0         매출액  3022314.00  2589355.00  3008709.00  3291027.00  790987.00  757883.00  791405.00  745663.00  860617.00  895006.00
1        영업이익   433766.00    65670.00   327260.00   401605.00   91834.00   64927.00   66853.00   46761.00  121661.00  167122.00
2       당기순이익   556541.00   154871.00   344514.00   398255.00  101009.00   775

---
## 4. ChatGPT / Gemini API로 분석

28차시에서 설정한 API를 사용하여 재무제표를 분석합니다.

In [24]:
from openai import OpenAI
from google import genai

openai_client = OpenAI()
gemini_client = genai.Client()

def analyze_with_openai(financial_text):
    """OpenAI API로 재무제표 분석 (Responses API)"""
    system_prompt, user_prompt = create_analysis_prompt(financial_text)

    try:
        response = openai_client.responses.create(
            model="gpt-5-mini",
            input=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
        )
        return response.output_text
    except Exception as e:
        return f"오류 발생: {str(e)}"

def analyze_with_gemini(financial_text):
    """Gemini API로 재무제표 분석"""
    system_prompt, user_prompt = create_analysis_prompt(financial_text)

    # Gemini는 system prompt를 user prompt에 포함
    full_prompt = f"""[시스템 지시사항]
{system_prompt}

[분석 요청]
{user_prompt}"""

    try:
        response = gemini_client.models.generate_content(
            model="gemini-2.5-flash",
            contents=full_prompt
        )
        return response.text
    except Exception as e:
        return f"오류 발생: {str(e)}"


def analyze_financial_statements(financial_text, api="openai"):
    """통합 재무제표 분석 함수"""
    if api == "openai":
        return analyze_with_openai(financial_text)
    elif api == "gemini":
        return analyze_with_gemini(financial_text)
    else:
        return "지원하지 않는 API입니다."

In [25]:
# 삼성전자 종합 분석 실행
print(f"[{USE_API.upper()} 재무제표 종합 분석 - 삼성전자]")
print("=" * 60)

result = analyze_financial_statements(samsung_text, USE_API)

display(Markdown(result))

[OPENAI 재무제표 종합 분석 - 삼성전자]


## 1. 수익성 분석
- 매출·영업이익·당기순이익 추이 평가  
  - 매출: 2022년 3,022,314억 → 2023년 2,589,355억(감소) → 2024년 3,008,709억(회복). 2025년 추정 3,291,027억으로 추가 성장 전망. 분기별로는 2025년 하반기(3Q·4QE)로 갈수록 매출·이익이 개선되는 모습을 보임(2025.3~9월: 79.1·74.6·86.1만 → 2025.12E 89.5만).  
  - 영업이익: 2022년 433,766억 → 2023년 65,670억(대폭 하락) → 2024년 327,260억(회복) → 2025E 401,605억(추정 추가 개선). 당기순이익도 유사한 흐름(556,541 → 154,871 → 344,514 → 398,255E).
- 영업이익률 및 순이익률 수준과 변화  
  - 영업이익률: 2022 14.35% → 2023 2.54% → 2024 10.88% → 2025E 12.20%. 분기별로는 2025 Q1~Q2에서 저조했으나 Q3·Q4E(14.14%, 18.67%)로 강한 개선.  
  - 순이익률: 2022 18.41% → 2023 5.98% → 2024 11.45% → 2025E 12.10%. 전반적으로 반도체 사이클 영향으로 변동성이 크나 2024~25년에 걸쳐 마진 회복세.

해석 요지: 2023년 반도체 부진으로 수익성이 크게 훼손되었으나 2024~25년에 걸친 실적 회복이 뚜렷. 다만 분기별 변동성(시즌성·사이클성)을 동반.

## 2. 안정성 분석
- 주요 지표  
  - 부채비율: 26.41%(2022) → 25.36%(2023) → 27.93%(2024). 보수적 수준(약 25~28%).  
  - 당좌비율(표시값): 211.68% → 189.46% → 187.80%로 높아 유동성 여유 존재(단기지급능력 양호).  
  - 유보율(누적이익 등): 매우 높음(약 41,000 수준), 내부유보가 풍부함.  
  - 현금흐름 관련 수치(직접 표기 없음) 대신 낮은 레버리지와 높은 당좌비율로 인해 재무완충 능력이 강한 것으로 판단.
- 재무구조 안정성 평가  
  - 재무 레버리지가 낮고 유동성 지표가 양호해 경기 후퇴나 투자확대(대규모 CAPEX) 시에도 대응 여력이 큼. 다만 반도체 등 설비투자·R&D 부담으로 현금소요가 클 수 있어 지속적 현금창출이 중요.

## 3. 성장성 분석
- 매출 및 이익 성장 추이  
  - 단기: 2023년 급감 후 2024년에 회복, 2025E 추가 성장 전망(매출·영업이익 모두 상승). 분기 흐름은 2025년 하반기 강세로 전환.  
  - 중기: AI·데이터센터 수요, 파운드리·시스템LSI·패키징 기술 투자, 모바일·가전 회복 등으로 수익원 다각화 및 수요 기반 확대 가능.
- 중장기 성장 가능성 평가  
  - 긍정 요인: 반도체(특히 고성능 메모리·파운드리)의 구조적 수요(AI 서버 등), 글로벌 제조·R&D 역량, 대규모 설비투자로 시장 지배력 유지 가능.  
  - 제약 요인: 메모리 가격·수요의 높은 경기민감성, 파운드리 경쟁(TSMC 등) 심화, 대규모 CAPEX에 따른 자본집약성. 중장기 성장성은 기술투자 성공과 제품 믹스 개선(고부가 제품 비중)에 달려 있음.

## 4. 투자 포인트
- 강점(2~3가지)  
  1) 견고한 재무체력(낮은 부채비율·높은 당좌비율)과 풍부한 내부유보로 경기 변동·대규모 투자에 대응 가능.  
  2) 시장지배력 및 스케일(메모리·파운드리·디바이스 전반)으로 제품·수요 회복 시 빠른 이익 반등 가능.  
  3) 안정적 배당정책(연간 배당 ~1,444원 수준, 배당수익률 2%대)으로 주주환원 지속.
- 리스크 요인(2~3가지)  
  1) 핵심 사업(특히 메모리) 사이클 민감성으로 실적 변동성이 큼(2023년 대폭 이익 축소 사례).  
  2) 경쟁 심화(파운드리·플랫폼 경쟁), 기술·가격 경쟁에 따른 마진 압력.  
  3) CAPEX·R&D 부담, 지정학 리스크(미·중 갈등 등)로 비용·공급망 리스크 존재.

## 5. 종합 의견
- 투자 관점 100자 이내 요약  
  견조한 재무·시장지위로 회복력 강하나 반도체 사이클·경쟁 리스크로 변동성 큼. 중장기 유망, 단기 타이밍 주의.

---
## 5. 기업 비교 분석

두 기업의 재무제표를 수집하여 비교 분석합니다.

In [26]:
# SK하이닉스 재무 데이터 수집
print("[SK하이닉스 재무 데이터 수집]")
print("=" * 60)

skhynix_data = get_financial_summary("000660")

if skhynix_data:
    print(f"\n회사명: {skhynix_data['company_name']}")
    print(f"종목코드: {skhynix_data['stock_code']}")
    print("\n[재무 요약 테이블]")
    display(skhynix_data['table'])
else:
    print("데이터 수집 실패")

[SK하이닉스 재무 데이터 수집]

회사명: SK하이닉스
종목코드: 000660

[재무 요약 테이블]


주요재무정보   최근 연간 실적                                    최근 분기 실적  \
       주요재무정보    2022.12    2023.12    2024.12 2025.12(E)    2024.09   
       주요재무정보     IFRS연결     IFRS연결     IFRS연결     IFRS연결     IFRS연결   
0         매출액  446216.00  327657.00  661930.00  945243.00  175731.00   
1        영업이익   68094.00  -77303.00  234673.00  438312.00   70300.00   
2       당기순이익   22417.00  -91375.00  197969.00  397979.00   57534.00   
3       영업이익률      15.26     -23.59      35.45      46.37      40.00   
4        순이익률       5.02     -27.89      29.91      42.10      32.74   
5   ROE(지배주주)       3.55     -15.61      31.06      42.91      17.31   
6        부채비율      64.12      87.52      62.15        NaN      65.95   
7        당좌비율      60.27      75.97     113.24        NaN      93.09   
8         유보율    1668.32    1397.12    1911.20        NaN    1703.85   
9      EPS(원)    3063.00  -12517.00   27182.00   54848.00    7897.00   
10     PER(배)      24.49     -11.30       6.40      13.78      12.19   
11     BPS(원)   92004.00   77752.00  107256.00  162471.00   94770.00   
12     PBR(배)       0.82       1.82       1.62       4.65       1.84   
13   주당배당금(원)    1200.00    1200.00    2204.00    1791.00     300.00   
14   시가배당률(%)       1.60       0.85       1.27        NaN       0.17   
15    배당성향(%)      37.01      -9.06       7.68        NaN       3.60   

                                                           
      2024.12    2025.03    2025.06    2025.09 2025.12(E)  
       IFRS연결     IFRS연결     IFRS연결     IFRS연결     IFRS연결  
0   197670.00  176391.00  222320.00  244489.00  301892.00  
1    80828.00   74405.00   92129.00  113834.00  157848.00  
2    80065.00   81082.00   69962.00  125975.00  142124.00  
3       40.89      42.18      41.44      46.56      52.29  
4       40.50      45.97      31.47      51.53      47.08  
5       31.06      37.94      39.27      43.20        NaN  
6       62.15      52.24      48.13      48.43        NaN  
7      113.24     104.14     117.87     134.90        NaN  
8     1911.20    2114.39    2298.84    2636.87        NaN  
9    10990.00   11136.00    9612.00   17301.00   17715.00  
10       6.40       5.34       7.37       7.09      42.67  
11  107256.00  117947.00  126196.00  144811.00  162471.00  
12       1.62       1.62       2.31       2.40       4.65  
13    1304.00     375.00     375.00     375.00        NaN  
14       0.75       0.20       0.13       0.11        NaN  
15      11.25       3.19       3.70       2.06        NaN

In [27]:
# SK하이닉스 데이터 변환
skhynix_text = format_for_llm(skhynix_data)

In [28]:
def compare_with_openai(company1_text, company2_text):
    """OpenAI로 두 기업 비교 분석"""

    system_prompt = """당신은 15년 경력의 증권 애널리스트입니다.
두 기업의 재무제표를 비교 분석하여 투자자에게 인사이트를 제공합니다.
객관적 데이터에 기반하여 분석하고, 한국어로 답변합니다."""

    user_prompt = f"""다음 두 기업의 재무제표를 비교 분석해주세요.

[기업 1]
{company1_text}

[기업 2]
{company2_text}

다음 형식으로 비교 분석 결과를 작성해주세요:

## 1. 요약 비교표
| 항목 | 기업1 | 기업2 | 비교 |
|------|-------|-------|------|
(주요 지표 비교)

## 2. 수익성 비교
- 영업이익률, 순이익률 비교

## 3. 안정성 비교
- 부채비율, 유동비율 비교

## 4. 투자 매력도 비교
- 각 기업의 강점/약점
- 어느 기업이 더 투자 가치가 있는지

## 5. 결론
- 100자 이내 요약
"""

    try:
        response = openai_client.responses.create(
            model="gpt-5-mini",
            input=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            max_output_tokens=2000
        )
        return response.output_text
    except Exception as e:
        return f"오류: {str(e)}"

def compare_with_gemini(company1_text, company2_text):
    """Gemini로 두 기업 비교 분석"""
    if not gemini_client:
        return None

    prompt = f"""당신은 15년 경력의 증권 애널리스트입니다.
두 기업의 재무제표를 비교 분석하여 투자자에게 인사이트를 제공합니다.
객관적 데이터에 기반하여 분석하고, 한국어로 답변합니다.

다음 두 기업의 재무제표를 비교 분석해주세요.

[기업 1]
{company1_text}

[기업 2]
{company2_text}

다음 형식으로 비교 분석 결과를 작성해주세요:

## 1. 요약 비교표
| 항목 | 기업1 | 기업2 | 비교 |
|------|-------|-------|------|
(주요 지표 비교)

## 2. 수익성 비교
## 3. 안정성 비교
## 4. 투자 매력도 비교
## 5. 결론 (100자 이내)
"""

    try:
        response = gemini_client.models.generate_content(
            model="gemini-2.5-flash",
            contents=prompt
        )
        return response.text
    except Exception as e:
        return f"오류: {str(e)}"


def compare_companies(company1_text, company2_text, api="openai"):
    """통합 기업 비교 함수"""
    if api == "openai":
        return compare_with_openai(company1_text, company2_text)
    elif api == "gemini":
        return compare_with_gemini(company1_text, company2_text)
    else:
        return "지원하지 않는 API입니다."

In [29]:
# 삼성전자 vs SK하이닉스 비교 분석
print(f"[{USE_API.upper()} 기업 비교 분석: 삼성전자 vs SK하이닉스]")
print("=" * 60)

comparison_result = compare_companies(samsung_text, skhynix_text, api=USE_API)

display(Markdown(comparison_result))

[OPENAI 기업 비교 분석: 삼성전자 vs SK하이닉스]


## 1. 요약 비교표
| 항목 | 삼성전자 (기업1, 2024) | SK하이닉스 (기업2, 2024) | 비교 |
|------|------------------------:|-------------------------:|------|
| 매출액 | 3,008,709 | 661,930 | 삼성 매출 규모 ≫ SK |
| 영업이익 | 327,260 | 234,673 | 삼성 절대이익 우세, SK 영업이익률 높음 |
| 당기순이익 | 344,514 | 197,969 | 삼성 순이익 절대 우세 |
| 영업이익률 | 10.88% | 35.45% | SK 수익성(마진) 훨씬 높음 |
| 순이익률 | 11.45% | 29.91% | SK 수익성 우위 |
| ROE(지배주주) | 9.03% | 31.06% | SK가 자본이익률 우수 |
| 부채비율 | 27.93% | 62.15% | 삼성이 재무레버리지 낮아 안정적 |
| 당좌비율(유동성) | 187.80% | 113.24% | 삼성 유동성 우수 |
| EPS(원) | 4,950원 | 27,182원 | SK 1주당 이익 높음 |
| PER(배) | 10.75 | 6.40 | SK가 이익 대비 저평가(또는 성장 반영) |
| BPS(원) | 57,981원 | 107,256원 | SK가 장부가치 우세 |
| PBR(배) | 0.92 | 1.62 | 삼성은 장부가 대비 저평가, SK는 프리미엄 |
| 주당배당금 | 1,446원 | 2,204원 | SK 현금배당 총액 높으나 배당성향 차이 존재 |
| 시가배당률 | 2.72% | 1.27% | 삼성이 배당수익률 높음 |
| 배당성향 | 29.18% | 7.68% | 삼성 배당정책이 더 우호적 |

(수치는 네이버 금융 제공 2024년 IFRS 연결 기준)

## 2. 수익성 비교
- 영업이익률: 삼성 10.9% vs SK 35.5%. SK가 메모리 호황으로 훨씬 높은 마진을 기록.  
- 순이익률: 삼성 11.5% vs SK 29.9%. SK는 영업레버리지와 한시적 요인(제품믹스·가격상승 등)으로 높은 순이익률을 확보.  
- 해석: SK는 사이클 성향이 강한 사업(메모리)으로 고수익·고변동성 구조, 삼성은 반도체·스마트폰·가전 등 다각화로 비교적 낮은 마진이지만 안정적 이익 기저를 갖춤.

## 3. 안정성 비교
- 부채비율: 삼성 27.9% vs SK 62.2% → 삼성의 재무안정성이 우수. SK는 레버리지 비중이 더 큼(그러나 2024년 실적 개선으로 재무지표 개선 중).  
- 당좌비율: 삼성 187.8% vs SK 113.2% → 삼성 유동성 여유가 큼. SK는 100% 초과로 단기지급능력은 확보했으나 삼성보다 여유는 적음.  
- 해석: 보수적 안전성을 중시하면 삼성 우위. SK는 높은 수익성으로 레버리지 감내 가능하나 경기하강 시 이익·현금흐름 급감 위험 존재.

## 4. 투자 매력도 비교
- 삼성전자 강점
  - 거대한 매출 규모와 사업 다각화(반도체·모바일·가전·디스플레이 등)로 안정적 현금흐름.
  - 낮은 부채비율·높은 당좌비율로 재무건전성 우수.
  - 안정적 배당(시가배당률·배당성향 상대적 우위)으로 인컴 투자에 적합.
  - PER·PBR 측면에서 밸류에이션 매력(저평가) 존재.
- 삼성전자 약점
  - SK 대비 ROE·마진이 낮아 주주수익률(단기 성장성) 측면에서는 매력도 제한.
  - 반도체 등 주력 사업의 주기적 변동 영향은 여전히 존재.
- SK하이닉스 강점
  - 높은 영업이익률·순이익률·ROE, 1주당 이익(EPS) 급증으로 성장성과 수익성 우수.
  - PER 낮아(이익 대비 주가 매력) 단기 수익 개선 시 주가 상승 여지 큼.
  - 장부가치(BPS) 대비 주당가치 상승 가능성.
- SK하이닉스 약점
  - 메모리 경기 민감도로 실적 변동성 큼(사이클 리스크).
  - 부채비율이 삼성보다 높아 경기 하강 시 부담 가능성.
  - 배당성향이 낮아 배당수익·주주환원 정책은 제한적.
- 어느 기업이 더 투자 가치 있는가?
  - 위험선호·성장추구 투자자: SK하이

---
## 학습 정리

### 1. 재무 분석 워크플로우
```
네이버 금융 크롤링 → 데이터 전처리 → 프롬프트 설계 → API 분석 → 결과 활용
```

### 2. API 사용법 비교

| 항목 | OpenAI | Gemini |
|------|--------|--------|
| 패키지 | `openai` | `google-genai` |
| 클라이언트 | `OpenAI()` | `genai.Client()` |
| 모델 | `gpt-5-mini` | `gemini-2.5-flash` |
| 호출 | `client.responses.create()` | `client.models.generate_content()` |
| 응답 | `response.output_text` | `response.text` |

### 3. 이전 차시 연계
| 차시 | 활용 내용 |
|------|----------|
| 15차시 | 네이버 금융 크롤링 (`pd.read_html`) |
| 27차시 | 프롬프트 엔지니어링 |
| 28차시 | OpenAI, Gemini API 설정 |

---

### 다음 차시 예고
- 30차시: [실습] LangChain으로 만드는 나만의 '투자 분석 봇'
  - LangChain 프레임워크 소개
  - RAG (검색 증강 생성) 개념
  - 외부 데이터 연동